In [63]:
%load_ext autoreload
%autoreload 2

import json
from datetime import datetime
import os
from pathlib import Path
import re
import sys

import pandas as pd
import polars as pl
import numpy as np

ROOT_PATH = Path(os.path.abspath("")).parents[1]

sys.path.append(str(ROOT_PATH))
from src.functions.json_values import (
    get_json_values,
    get_json_values_parallel,
    select_value,
)
from src.functions.flatten_dict import flatten_dict

CLASSIFIED_PATH = ROOT_PATH / "data/classified"
SCRAPING_DAY = 22

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
# Extract date of classified
def get_day(file_name: str):
    ymd_groups = r"^classified_.+_(\d+)_(\d+).json$"
    year_string = re.findall(ymd_groups, file_name)[0][
        0
    ]  # First element of list, first group
    return datetime.strptime(year_string, "%Y%m%d").day

In [65]:
# Keep classified files that were captured on the 15th of May
list_filenames = os.listdir(CLASSIFIED_PATH)
new_list_filenames = []

for filename in list_filenames:
    try:
        filename_day = get_day(filename)
    except IndexError:
        print(
            filename
        )  # Some filenames are changed by mistakes <- shouldn't happen too often
        continue
    if filename_day == SCRAPING_DAY:
        new_list_filenames.append(filename)

list_filenames = new_list_filenames

classified_10000290apartmentforrent8600Diksmuide_20230522_212157.json~


In [66]:
# Read one file and extract useful variables
with open(CLASSIFIED_PATH / list_filenames[0]) as json_file:
    class_dict = json.load(json_file)

class_dict.keys()

dict_keys(['user', 'classified', 'customer', 'screen', 'other_defined', 'classified_table'])

In [67]:
# Find and select renting locations
list_transactions = get_json_values(
    list_filenames, CLASSIFIED_PATH, "classified", "transactionType"
)

In [68]:
# Get unique
np.unique(np.array(list_transactions))

array(['for rent', 'for sale'], dtype='<U8')

In [69]:
## Filter for rent and apartments
list_types = get_json_values(list_filenames, CLASSIFIED_PATH, "classified", "type")

In [70]:
np.array(np.unique(list_types))

array(['apartment', 'apartment group', 'house', 'house group'],
      dtype='<U15')

In [71]:
## Filter JSONS about apartment to rent
bl_type = np.isin(np.array(list_types), ["apartment", "apartment group"])
bl_transact = np.isin(np.array(list_transactions), "for rent")
bl = np.logical_and(bl_type, bl_transact)

list_apartrent = [filename for filename, b in zip(list_filenames, bl) if b]

In [72]:
# Possible values
pl.DataFrame(flatten_dict(select_value(list_apartrent[0], CLASSIFIED_PATH))).shape

(1, 75)

In [73]:
select_value(list_apartrent[0], CLASSIFIED_PATH)

{'user': {'loginStatus': 'logged out', 'id': '', 'personal': {'language': ''}},
 'classified': {'id': '10578471',
  'type': 'apartment',
  'subtype': 'apartment',
  'price': '1050',
  'transactionType': 'for rent',
  'zip': '1060',
  'visualisationOption': 'xl',
  'kitchen': {'type': 'installed'},
  'building': {'constructionYear': '1929', 'condition': 'good'},
  'energy': {'heatingType': 'gas'},
  'certificates': {'primaryEnergyConsumptionLevel': '324'},
  'bedroom': {'count': '2'},
  'land': {'surface': ''},
  'atticExists': '',
  'basementExists': '',
  'outdoor': {'garden': {'surface': ''}, 'terrace': {'exists': ''}},
  'specificities': {'SME': {'office': {'exists': 'true'}}},
  'wellnessEquipment': {'hasSwimmingPool': ''},
  'parking': {'parkingSpaceCount': {'indoor': '', 'outdoor': ''}},
  'condition': {'isNewlyBuilt': ''}},
 'customer': {'id': '2575488',
  'name': 'we invest bruxelles',
  'family': 'agency',
  'groupInfo': {'id': '', 'name': ''},
  'networkInfo': {'id': '', 'nam

In [74]:
# Defining component I want to extract from the apartments to rent
component_dict = {
    "listing_id": ["classified", "id"],
    "price": ["classified", "price"],
    "contact_type": ["customer", "family"],
    "contact_name": ["customer", "name"],
    "subtype": ["classified", "subtype"],
    "zip_code": ["classified", "zip"],
    "construction_year": ["building", "constructionYear"],
    "building_condition": ["building", "condition"],
    "energy_consumption": ["certificates", "primaryEnergyConsumptionLevel"],
    "energy_class": ["classified_table", "energy_class"],
    "n_bedrooms": ["bedroom", "count"],
    "land_surface": ["land", "surface"],
    "surroundings_type": ["classified_table", "surroundings_type"],
    "living_area": ["classified_table", "living_area"],
}

In [75]:
pl_apart = pl.DataFrame()

for key in component_dict.keys():
    data_arr = get_json_values(list_apartrent, CLASSIFIED_PATH, *component_dict[key])
    pl_apart = pl_apart.with_columns(pl.lit(data_arr).alias(key))


pl_apart.head()

listing_id,price,contact_type,contact_name,subtype,zip_code,construction_year,building_condition,energy_consumption,energy_class,n_bedrooms,land_surface,surroundings_type,living_area
str,str,str,str,str,str,f32,f32,f32,str,f32,f32,str,str
"""10578471""","""1050""","""agency""","""we invest brux…","""apartment""","""1060""",null,null,null,"""f""",null,null,"""urban""","""92"""
"""10579694""","""2750""","""agency""","""bathim &amp; c…","""penthouse""","""1050""",null,null,null,"""d""",null,null,"""urban""","""200"""
"""10578602""","""1250""","""agency""","""bathim &amp; c…","""apartment""","""1000""",null,null,null,"""b""",null,null,"""living area (r…","""50"""
"""10527337""","""1350""","""agency""","""place 4 you""","""duplex""","""1150""",null,null,null,"""g""",null,null,"""living area (r…","""80"""
"""10578603""","""1250""","""agency""","""bathim &amp; c…","""apartment""","""1030""",null,null,null,"""d""",null,null,"""isolated""","""85"""


In [76]:
# With price, save only max price, parse
price_list = pl_apart.select("price").to_numpy().ravel().tolist()

In [77]:
# Investigating strange price patterns
pattern = r"(\d+)(.*)"
for price in price_list:
    found = re.findall(pattern, price)
    try:
        if found[0][1] != "":
            print(found[0])
    except IndexError:
        print(found)

('900', ' - 922')
('950', ' - 1050')
('950', ' - 1390')
('915', ' - 915')
('860', ' - 1180')
[]
('825', ' - 995')
('765', ' - 853')
('950', ' - 1150')


In [81]:
adjusted_prices = []

pattern = r"(\d+)( - )?(\d+)"
for price in price_list:
    found = re.findall(pattern, price)
    try:
        if found[0][1] != "":
            adjusted_prices.append(found[0][2])
        else:
            adjusted_prices.append(price)
    except IndexError:
        adjusted_prices.append(None)

In [83]:
# Replace adjusted prices (we retain the max rent price)
pl_apart = pl_apart.with_columns(pl.lit(adjusted_prices).cast(pl.Int64).alias("price"))

In [84]:
# Cast rows as relevant
q = [
    pl.col("price").cast(pl.Int64),
    pl.col("n_bedrooms").cast(pl.Int64),
    pl.col("energy_consumption").cast(pl.Int64),
    pl.col("construction_year").cast(pl.Int64),
]

pl_apart = pl_apart.with_columns(q)